In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install transformers
!pip install sentencepiece
!pip install emoji

     |████████████████████████████████| 1.4MB 5.4MB/s 
     |████████████████████████████████| 2.9MB 60.8MB/s 
     |████████████████████████████████| 890kB 52.0MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=7352dc589a307b6d7d973f5c2effbc2d45ea3ac95abc5f3a388c3dabb590e32d
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
     |████████████████████████████████| 1.1MB 5.3MB/s 
     |████████████████████████████████| 51kB 2.7MB/s 
  Created wheel for emoji: filename=emoji-0.6.0-cp36-none-any.whl size=49716 sha256=bd2d3a8323708928d90497fcb3f485c15d168807734c8c4da07741b29a4f7a09
  Stored in directory: /root/.cache/pip/wheels/46/2c/8b/9dcf5216ca68e14e0320e283692dce8ae321cdc01e73e17796
Successfully built emoji


In [3]:
import warnings

warnings.simplefilter("ignore", UserWarning)
warnings.simplefilter("ignore", FutureWarning)
warnings.simplefilter("ignore", DeprecationWarning)

import tensorflow as tf
import torch
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader, RandomSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from transformers import XLMRobertaTokenizer, XLMRobertaForSequenceClassification, AdamW, get_linear_schedule_with_warmup
from tqdm import tqdm, trange
import pandas as pd
import io
import os
import numpy as np
from sklearn.metrics import f1_score, accuracy_score
from statistics import mode

import re
import emoji
import random

import nltk
nltk.download('stopwords')
from nltk import word_tokenize, pos_tag
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import sent_tokenize, TweetTokenizer
from nltk.corpus import wordnet, stopwords

# specify GPU device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
torch.cuda.get_device_name(0)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


'Tesla V100-SXM2-16GB'

In [4]:
def preprocess(df):
    
    #removes URL
    pattern = r'https.?://[^\s]+[\s]?'
    df["tweet"] = df["tweet"].str.replace(pat=pattern, repl="", regex=True)
    
    #removes usernames/mentions
    pattern = r'@[^\s]+'
    df["tweet"] = df["tweet"].str.replace(pat=pattern, repl="", regex=True)
    
    #removes emoji and smiley
    pattern = re.compile("["
                         u"\U0001F600-\U0001F64F"
                         u"\U0001F300-\U0001F5FF"
                         u"\U0001F680-\U0001F6FF"
                         u"\U0001F1E0-\U0001F1FF"
                         u"\U00002500-\U00002BEF"
                         u"\U00002702-\U000027B0"
                         u"\U00002702-\U000027B0"
                         u"\U000024C2-\U0001F251"
                         u"\U0001f926-\U0001f937"
                         u"\U00010000-\U0010ffff"
                         u"\u2640-\u2642"
                         u"\u2600-\u2B55"
                         u"\u200d"
                         u"\u23cf"
                         u"\u23e9"
                         u"\u231a"
                         u"\ufe0f"
                         u"\u3030"
                         "]+", flags=re.UNICODE)
    df["tweet"] = df["tweet"].str.replace(pat=pattern, repl="", regex=True)
    
    #removes numbers
    pattern = r'\d+'
    df["tweet"] = df["tweet"].str.replace(pat=pattern, repl="", regex=True)
    
    #removes punctuation
    pattern = r"[^\w\s]"
    df["tweet"] = df["tweet"].str.replace(pat=pattern, repl=" ", regex=True)

    #removes stop words
    stop_words = stopwords.words("english")    
    remove_stop_words = lambda row: " ".join([token for token in row.split(" ")
                                              if token not in stop_words])
    df["tweet"] = df["tweet"].apply(remove_stop_words)
    
    #removes extra spaces
    pattern = r"[\s]+"
    df["tweet"] = df["tweet"].str.replace(pat=pattern, repl=" ", regex=True)
    
    return(df)

In [5]:
def tokenize_data(df, sampling=False):
    sentences = ["[CLS] " + query + " [SEP]" for query in df['tweet']]
    tokenizer = XLMRobertaTokenizer.from_pretrained('xlm-roberta-base')
    tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]
    
    MAX_LEN = 100
    input_ids = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts],
                          maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
    
    labels = df['label'].copy()

    # Create attention masks
    attention_masks = []
    for seq in input_ids:
        seq_mask = [float(i>0) for i in seq]
        attention_masks.append(seq_mask)
    return input_ids, attention_masks, labels

def Data_Loader(inputs_ids, attention_masks, labels, batch_size=3): 
    data = TensorDataset(torch.LongTensor(inputs_ids), torch.LongTensor(attention_masks), torch.LongTensor(labels.ravel()))
    sampler = RandomSampler(data)
    dataloader = DataLoader(data, sampler=sampler, batch_size=batch_size)
    return dataloader

In [6]:
def model_test(model,prediction_dataloader):
    model.eval()
    predictions , true_labels = [], []
    for batch in prediction_dataloader:
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        with torch.no_grad():
            logits = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)[0]
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        predictions+=list(np.argmax(logits, axis=1).flatten())
        # true_labels+=list(label_ids.flatten())
    return predictions

In [7]:
def model_initialise(path= None , use_saved_model=False):
  model = XLMRobertaForSequenceClassification.from_pretrained("xlm-roberta-base", num_labels = 2, output_attentions = False, output_hidden_states = False).cuda()

  if(use_saved_model==True):
    checkpoint = torch.load(path)
    model.load_state_dict(checkpoint['model_state_dict'])
    # optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

  return model

In [8]:
from io import StringIO 
test_string = """tweet,label
                 Click below for a FREE download of a colorfully illustrated 132 page e-book on the Zionist-engineered INTENTIONAL destruction of Western civilization, 1
                 In my opinion using cuss words like bitch is demeaning to women, 0
                 I hope they all starve, 1"""

data = io.StringIO(test_string)
test_df = pd.read_csv(data, sep=",")

In [9]:
model = model_initialise('/content/drive/My Drive/XLMRMultilingual models/XLMREnglish.pth',use_saved_model=True)
test_input_ids, test_attention_masks, test_labels = tokenize_data(test_df)
test_dataloader = Data_Loader(test_input_ids, test_attention_masks, test_labels)
pred = model_test(model, test_dataloader)
print(test_df, pred)

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense


                                               tweet  label
0                   Click below for a FREE downlo...      1
1                   In my opinion using cuss word...      0
2                             I hope they all starve      1 [0, 0, 0]


In [ ]:
from io import StringIO 
test_string = """tweet,label
                 Click below for a FREE download of a colorfully illustrated 132 page e-book on the Zionist-engineered INTENTIONAL destruction of Western civilization, 1
                 In my opinion using cuss words like bitch is demeaning to women, 0
                 I hope they all starve, 1"""

data = io.StringIO(test_string)
df = pd.read_csv(data, sep=",")
print(df.tweet[0])
test_df = preprocess(df)
print(test_df.tweet[0])

                 Click below for a FREE download of a colorfully illustrated 132 page e-book on the Zionist-engineered INTENTIONAL destruction of Western civilization
 Click FREE download colorfully illustrated page e book Zionist engineered INTENTIONAL destruction Western civilization


In [ ]:
model = model_initialise('/content/drive/My Drive/XLMRMultilingual models/XLMREnglish.pth',use_saved_model=True)
test_input_ids, test_attention_masks, test_labels = tokenize_data(test_df)
test_dataloader = Data_Loader(test_input_ids, test_attention_masks, test_labels)
pred = model_test(model, test_dataloader)
print(test_df, pred)

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense

                                               tweet  label
0   Click FREE download colorfully illustrated pa...      1
1   In opinion using cuss words like bitch demean...      0
2                                      I hope starve      1 [0, 0, 1]
